In [ ]:
from dotenv import load_dotenv
import os
import pickle

load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [ ]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

### Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLm, it is important to understand a new concept and that of a "tool"


Agent makes use of ReACt framework. ReAct = Reasoning + Action

### Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents

In [ ]:
from langchain.agents import load_tools
from langchain.llms import OpenAI

llm = OpenAI(model_name = "gpt-4", temperature = 0.7)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm = llm)
tools

In [ ]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Math Tool",
        func = tools[0].run,
        description= "tool to calculate, nothing else"
    )
]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    tool_list,
    llm,
    agent = "zero-shot-react-description",
    verbose = True)
agent.run("How are you?")

In [ ]:
agent.run("what is 100 devided by 25")

We should write our custom tools that retrive information from our vector database

### Custom Tool

You can also create your own tools by creatig a class that inherits from BaseTool class.

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

# our class is inherited from BaseTool class
class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        # we have create retriever for query
        store = vectorstore.as_retriever()
        # use retriever to get relevant documents and pass in the query (this will be embedded and we will get back the most similar documents from the document store and this will be list of documents)
        docs = store.get_relevant_documents(query)
        # we can extract page content from document class
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    
    # for asynchronous tasks
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        # Use the tool asynchronously
        raise NotImplementedError("custom search does not support async")

In [ ]:
from langchain.agents import AgentType

# tool name is the class name that we defined
tools = [CustomSearchTool()]

# pass this as a tool list to our initialize agent function
agent = initialize_agent(tools,llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [ ]:
agent.run("when does the restaurant open?")